In [4]:
!pip install -q segmentation-models-pytorch roboflow pycocotools albumentations tqdm

In [5]:
import warnings

warnings.filterwarnings("ignore", message="AssertionError: can only test a child process")
warnings.filterwarnings("ignore", message="^can only test a child process^")
warnings.filterwarnings("ignore", message="can only test a child process")
warnings.filterwarnings("ignore", message="_MultiProcessingDataLoaderIter")

In [6]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from tqdm.auto import tqdm
import random
from roboflow import Roboflow

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

In [ ]:
from google.colab import userdata

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')

rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Regional Dataset
project = rf.workspace("workspace-ou3p7").project("floorplancis_fpc")
version = project.version(6)
dataset = version.download("coco-segmentation")

# Cubicasa-5k Dataset
# project = rf.workspace("workspace-ou3p7").project("cubicasa-he2yn-bewqv")
# version = project.version(1)
# dataset = version.download("coco-segmentation")

dataset_location = dataset.location
train_dir, train_annot = os.path.join(dataset_location, "train"), os.path.join(dataset_location, "train", "_annotations.coco.json")
valid_dir, valid_annot = os.path.join(dataset_location, "valid"), os.path.join(dataset_location, "valid", "_annotations.coco.json")
test_dir, test_annot = os.path.join(dataset_location, "test"), os.path.join(dataset_location, "test", "_annotations.coco.json")

In [8]:
class COCOSegmentationDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transforms=None):
        self.root_dir, self.transforms = root_dir, transforms
        self.coco = COCO(annotation_file)
        self.img_ids = self.coco.getImgIds()

        self.wall_cat_id = self.coco.getCatIds(catNms=['wall'])
        self.door_cat_id = self.coco.getCatIds(catNms=['door'])
        self.window_cat_id = self.coco.getCatIds(catNms=['window'])
        print(f"ID found for classes: Wall={self.wall_cat_id}, Door={self.door_cat_id}, Window={self.window_cat_id}")

    def __len__(self):
        return len(self.img_ids)

    # Creates a mask for openings by calculating the geometry of a larger bounding box and drawing it directly.
    def _create_thick_openings_mask(self, anns, h, w):
            openings_mask = np.zeros((h, w), dtype=np.uint8)

            for ann in anns:
                cat_id = ann['category_id']
                if cat_id in self.door_cat_id or cat_id in self.window_cat_id:
                    single_opening_mask = self.coco.annToMask(ann)

                    _, binary_mask = cv2.threshold(single_opening_mask, 0, 255, cv2.THRESH_BINARY)
                    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    if not contours: continue

                    rect = cv2.minAreaRect(contours[0])

                    center = rect[0]
                    size = rect[1]
                    angle = rect[2]

                    thickness_to_add = 30
                    if size[0] < size[1]:
                        new_size = (size[0] + thickness_to_add, size[1])
                    else:
                        new_size = (size[0], size[1] + thickness_to_add)

                    new_rect = (center, new_size, angle)
                    new_box_points = np.intp(cv2.boxPoints(new_rect))

                    cv2.drawContours(openings_mask, [new_box_points], 0, (255), -1)

            return openings_mask

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root_dir, img_info['file_name'])
        image = cv2.imread(img_path)

        if image is None:
            return self.__getitem__((idx + 1) % len(self))

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        h, w = img_info['height'], img_info['width']
        wall_mask = np.zeros((h, w), dtype=np.uint8)

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        for ann in anns:
            if ann['category_id'] in self.wall_cat_id:
                wall_mask = np.maximum(wall_mask, self.coco.annToMask(ann))

        thick_openings_mask = self._create_thick_openings_mask(anns, h, w)
        final_mask = cv2.subtract(wall_mask, thick_openings_mask)

        kernel = np.ones((5,5), np.uint8)
        final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_CLOSE, kernel)

        image_h, image_w, _ = image.shape
        mask_h, mask_w = final_mask.shape
        if image_h != mask_h or image_w != mask_w:
            final_mask = cv2.resize(final_mask, (image_w, image_h), interpolation=cv2.INTER_NEAREST)

        if self.transforms:
            augmented = self.transforms(image=image, mask=final_mask)
            image, mask = augmented['image'], augmented['mask']
        else:
            mask = final_mask

        return image, mask

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 512, 512

train_transforms = A.Compose([
    A.Affine(
        scale=(0.9, 1.1),
        translate_percent=(-0.05, 0.05),
        rotate=(-15, 15),
        p=0.7,
        border_mode=cv2.BORDER_CONSTANT,
    ),
    A.Perspective(p=0.3),
    A.OneOf([
        A.ElasticTransform(p=1.0, alpha=120, sigma=120 * 0.05),
        A.GridDistortion(p=1.0),
    ], p=0.2),

    A.RandomBrightnessContrast(p=0.5),
    A.CLAHE(p=0.2),
    A.OneOf([
        A.GaussNoise(p=1.0),
        A.ISONoise(p=1.0),
    ], p=0.3),

    A.Resize(IMG_WIDTH, IMG_HEIGHT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

valid_transforms = A.Compose([
    A.Resize(IMG_WIDTH, IMG_HEIGHT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

from torch.utils.data import ConcatDataset, random_split, Subset

raw_train_dataset = COCOSegmentationDataset(train_dir, train_annot, transforms=None)
raw_valid_dataset = COCOSegmentationDataset(valid_dir, valid_annot, transforms=None)
raw_test_dataset = COCOSegmentationDataset(test_dir, test_annot, transforms=None)

full_dataset = ConcatDataset([raw_train_dataset, raw_valid_dataset, raw_test_dataset])
print(f"Total images: {len(full_dataset)}")

train_size = int(0.8 * len(full_dataset))
valid_size = len(full_dataset) - train_size

train_subset, valid_subset = random_split(full_dataset, [train_size, valid_size],
                                          generator=torch.Generator().manual_seed(42))

class TransformedSubset(Dataset):
    def __init__(self, subset, transforms):
        self.subset = subset
        self.transforms = transforms

    def __getitem__(self, idx):
        image, mask = self.subset[idx]
        if self.transforms:
            augmented = self.transforms(image=np.array(image), mask=mask)
            return augmented['image'], augmented['mask']
        return image, mask

    def __len__(self):
        return len(self.subset)

train_dataset = TransformedSubset(train_subset, transforms=train_transforms)
valid_dataset = TransformedSubset(valid_subset, transforms=valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=2, pin_memory=True)

print("-" * 30)
print(f"Images in train_loader: {len(train_loader.dataset)}")
print(f"Images in valid_loader: {len(valid_loader.dataset)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_original_images(dataset, rows=3, cols=4):
    num_images = rows * cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    fig.suptitle("Original Dataset Images", fontsize=20)

    axes = axes.flatten()

    for i in range(min(num_images, len(dataset))):
        image, _ = dataset[i]

        ax = axes[i]
        ax.imshow(image)
        ax.set_title(f"Image {i}")
        ax.axis('off')

    for j in range(num_images, len(axes)):
        axes[j].axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

visualize_original_images(full_dataset)

In [ ]:
import matplotlib.pyplot as plt

def visualize_samples(dataset, num_samples=4):
    fig, axes = plt.subplots(num_samples, 2, figsize=(10, num_samples * 5))
    fig.suptitle("Dataset images and masks", fontsize=16)

    indices = np.random.choice(len(dataset), num_samples, replace=False)

    for i, idx in enumerate(indices):
        image, mask = dataset[idx]
        image = image.permute(1, 2, 0).numpy()

        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

        ax_img = axes[i, 0]
        ax_img.imshow(image)
        ax_img.set_title(f"Image {idx}")

        ax_mask = axes[i, 1]
        ax_mask.imshow(mask.numpy(), cmap='gray')
        ax_mask.set_title(f"Mask {idx}")

        for ax in [ax_img, ax_mask]:
            ax.set_xticks([])
            ax.set_yticks([])

            for spine in ax.spines.values():
                spine.set_edgecolor('black')
                spine.set_linewidth(1.5)
                spine.set_visible(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

visualize_samples(train_dataset, num_samples=4)

In [12]:
import time

DEVICE = "cuda"

def train_epoch(model, loader, loss_fn, optimizer, device):
    model.train()
    running_loss, total_iou = 0.0, 0.0

    epoch_tp = torch.tensor(0.0, device='cpu')
    epoch_fp = torch.tensor(0.0, device='cpu')
    epoch_fn = torch.tensor(0.0, device='cpu')
    epoch_tn = torch.tensor(0.0, device='cpu')

    pbar = tqdm(loader, desc="Training")
    for images, masks in pbar:
        images, masks = images.to(device), masks.to(device).unsqueeze(1).float()

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        pred_probs = torch.sigmoid(outputs)
        pred_masks = (pred_probs > 0.5).float()

        tp, fp, fn, tn = smp.metrics.get_stats(pred_masks.long(), masks.long(), mode='binary')

        epoch_tp += tp.sum().detach().cpu()
        epoch_fp += fp.sum().detach().cpu()
        epoch_fn += fn.sum().detach().cpu()
        epoch_tn += tn.sum().detach().cpu()

        pbar.set_postfix(loss=f"{loss.item():.4f}")

    epoch_iou = smp.metrics.iou_score(epoch_tp, epoch_fp, epoch_fn, epoch_tn)

    return running_loss / len(loader.dataset), epoch_iou.item()

def valid_epoch(model, loader, loss_fn, device):
    model.eval()
    running_loss = 0.0

    epoch_tp = torch.tensor(0.0, device='cpu')
    epoch_fp = torch.tensor(0.0, device='cpu')
    epoch_fn = torch.tensor(0.0, device='cpu')
    epoch_tn = torch.tensor(0.0, device='cpu')

    torch.cuda.reset_peak_memory_stats(device)
    torch.cuda.synchronize(device)
    start_time = time.perf_counter()

    with torch.no_grad():
        pbar = tqdm(loader, desc="Validation")
        for images, masks in pbar:
            images, masks = images.to(device), masks.to(device).unsqueeze(1).float()

            outputs = model(images)
            loss = loss_fn(outputs, masks)

            running_loss += loss.item() * images.size(0)

            pred_probs = torch.sigmoid(outputs)
            pred_masks = (pred_probs > 0.5).float()

            tp, fp, fn, tn = smp.metrics.get_stats(pred_masks.long(), masks.long(), mode='binary')

            epoch_tp += tp.sum().detach().cpu()
            epoch_fp += fp.sum().detach().cpu()
            epoch_fn += fn.sum().detach().cpu()
            epoch_tn += tn.sum().detach().cpu()

            pbar.set_postfix(loss=f"{loss.item():.4f}")

    torch.cuda.synchronize(device)
    end_time = time.perf_counter()

    # VRAM
    peak_vram_mb = torch.cuda.max_memory_allocated(device) / (1024 ** 2)

    # FPS
    total_time = end_time - start_time
    total_images = len(loader.dataset)
    fps = total_images / total_time

    # Metrics
    epoch_iou = smp.metrics.iou_score(epoch_tp, epoch_fp, epoch_fn, epoch_tn)
    epoch_loss = running_loss / total_images

    return epoch_loss, epoch_iou.item(), fps, peak_vram_mb

**Training**

Testing SOTA architectures compared to the hybrid

In [ ]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import pandas as pd
import time
import gc
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 30
LR = 0.0001

if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"Running on: {DEVICE}")

LOSS_FUNCTIONS = {
    "Dice": smp.losses.DiceLoss(mode='binary'),
    "Focal": smp.losses.FocalLoss(mode='binary'),
    "Lovasz": smp.losses.LovaszLoss(mode='binary'),
    "Tversky": smp.losses.TverskyLoss(mode='binary', alpha=0.6, beta=0.4, gamma=1.0)
}

def get_model(model_name, encoder_name, hybrid=False):
    params = {
        "encoder_name": encoder_name,
        "encoder_weights": "imagenet",
        "in_channels": 3,
        "classes": 1,
    }

    if model_name == "Unet":
        if hybrid:
            segformer_full = smp.Segformer(encoder_name=encoder_name, encoder_weights="imagenet")
            model = smp.Unet(
                decoder_attention_type="scse",
                **{**params, "encoder_weights": None}
            )
            model.encoder = segformer_full.encoder
        else:
            model = smp.Unet(decoder_attention_type="scse", **params)
    elif model_name == "UnetPlusPlus":
        model = smp.UnetPlusPlus(decoder_attention_type="scse", **params)
    elif model_name == "DeepLabV3Plus":
        model = smp.DeepLabV3Plus(**params)
    elif model_name == "UperNet":
        model = smp.UPerNet(**params)
    elif model_name == "FPN":
        model = smp.FPN(**params)
    elif model_name == "Segformer":
        model = smp.Segformer(**params)
    else:
        raise ValueError(f"Unknown model architecture: {model_name}")

    return model.to(DEVICE)

def train_epoch(model, loader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0

    for images, masks in loader:
        images = images.to(device)
        masks = masks.to(device).unsqueeze(1).float()

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def evaluate_metrics(model, loader, device):
    model.eval()
    tp_tot, fp_tot, fn_tot, tn_tot = 0, 0, 0, 0

    with torch.no_grad():
        for images, masks in loader:
            images = images.to(device)
            masks = masks.to(device).unsqueeze(1).float()

            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).long()
            masks = masks.long()

            tp, fp, fn, tn = smp.metrics.get_stats(preds, masks, mode='binary')
            tp_tot += tp.sum()
            fp_tot += fp.sum()
            fn_tot += fn.sum()
            tn_tot += tn.sum()

    iou = smp.metrics.iou_score(tp_tot, fp_tot, fn_tot, tn_tot, reduction='micro').item()
    recall = smp.metrics.recall(tp_tot, fp_tot, fn_tot, tn_tot, reduction='micro').item()
    precision = smp.metrics.precision(tp_tot, fp_tot, fn_tot, tn_tot, reduction='micro').item()
    accuracy = smp.metrics.accuracy(tp_tot, fp_tot, fn_tot, tn_tot, reduction='micro').item()

    return iou, recall, precision, accuracy

def benchmark_speed_memory(model, loader, device):
    model.eval()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

    start_time = time.time()
    total_frames = 0

    with torch.no_grad():
        for images, _ in loader:
            images = images.to(device)
            _ = model(images)
            total_frames += images.size(0)

    torch.cuda.synchronize()
    end_time = time.time()

    fps = total_frames / (end_time - start_time)
    vram_usage = torch.cuda.max_memory_allocated() / (1024 ** 2) # MiB

    return fps, vram_usage

experiments_config = [
    {
        "model": "UperNet",
        "encoder": "mit_b4",
        "hybrid": False,
        "tag": "UperNet (MiT-B4)"
    },
    {
        "model": "Unet",
        "encoder": "mit_b4",
        "hybrid": True,
        "tag": "MitUnet"
    },
    {
        "model": "Unet",
        "encoder": "resnet50",
        "hybrid": False,
        "tag": "Unet (scSE)"
    },
        {
        "model": "UnetPlusPlus",
        "encoder": "resnet50",
        "hybrid": False,
        "tag": "Unet++"
    },
    {
        "model": "DeepLabV3Plus",
        "encoder": "resnet50",
        "hybrid": False,
        "tag": "DeepLabV3+"
    },
    {
        "model": "Segformer",
        "encoder": "mit_b4",
        "hybrid": False,
        "tag": "Segformer"
    },
]

results_data = []

print(f"\nStart Comparative Benchmark: {len(experiments_config)} models x {len(LOSS_FUNCTIONS)} losses")

for config in experiments_config:
    for loss_name, loss_fn in LOSS_FUNCTIONS.items():

        experiment_name = f"{config['tag']} + {loss_name}"
        print(f"Training: {experiment_name}")

        model = get_model(config["model"], config["encoder"], config["hybrid"])
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

        best_iou = 0.0

        for epoch in range(1, EPOCHS + 1):
            train_loss = train_epoch(model, train_loader, loss_fn, optimizer, DEVICE)

            iou, recall, precision, accuracy = evaluate_metrics(model, valid_loader, DEVICE)
            scheduler.step(iou)

            if iou > best_iou:
                best_iou = iou

            print(f"\rEpoch {epoch}/{EPOCHS} | Loss: {train_loss:.4f} | IoU: {iou:.4f}", end="")
        print(f"\nTraining finished. Best IoU during train: {best_iou:.4f}")

        print("Running final evaluation and benchmarks...")
        final_iou, final_recall, final_precision, final_accuracy = evaluate_metrics(model, valid_loader, DEVICE)
        fps, vram = benchmark_speed_memory(model, valid_loader, DEVICE)

        results_data.append({
            "Model Type": config["tag"],
            "Model Arch": config["model"],
            "Encoder": config["encoder"],
            "Loss Function": loss_name,
            "IoU": round(final_iou, 4),
            "Recall": round(final_recall, 4),
            "Precision": round(final_precision, 4),
            "Accuracy": round(final_accuracy, 4),
            "FPS": round(fps, 2),
            "VRAM (MiB)": round(vram, 2)
        })

        del model, optimizer, scheduler
        torch.cuda.empty_cache()
        gc.collect()

df_results = pd.DataFrame(results_data)
df_results = df_results.sort_values(by="IoU", ascending=False).reset_index(drop=True)

print("\n\nCOMPARATIVE TABLE")
print(df_results.to_string())

df_results.to_excel("model_comparison_results.xlsx", index=False)

Testing Tversky loss

In [ ]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import pandas as pd
import time
import gc
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

DEVICE = "cuda"
EPOCHS = 30
LR = 0.0001
BATCH_SIZE = 4

TVERSKY_ALPHAS = [0.6, 0.7, 0.8, 0.9]

if torch.cuda.is_available():
    torch.cuda.empty_cache()


def get_hybrid_model(encoder_name="mit_b4"):
    segformer_full = smp.Segformer(
        encoder_name=encoder_name,
        encoder_weights="imagenet"
    )
    model = smp.Unet(
        encoder_name=encoder_name,
        encoder_weights=None,
        in_channels=3,
        classes=1,
        decoder_attention_type="scse"
    )

    model.encoder = segformer_full.encoder

    return model.to(DEVICE)

def train_epoch(model, loader, loss_fn, optimizer, device):
    model.train()
    total_loss = 0

    for images, masks in loader:
        images = images.to(device)
        masks = masks.to(device).unsqueeze(1).float()

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def evaluate_metrics(model, loader, device):
    model.eval()
    tp_tot, fp_tot, fn_tot, tn_tot = 0, 0, 0, 0

    with torch.no_grad():
        for images, masks in loader:
            images = images.to(device)
            masks = masks.to(device).unsqueeze(1).float()

            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).long()
            masks = masks.long()

            tp, fp, fn, tn = smp.metrics.get_stats(preds, masks, mode='binary')
            tp_tot += tp.sum()
            fp_tot += fp.sum()
            fn_tot += fn.sum()
            tn_tot += tn.sum()

    iou = smp.metrics.iou_score(tp_tot, fp_tot, fn_tot, tn_tot, reduction='micro').item()
    recall = smp.metrics.recall(tp_tot, fp_tot, fn_tot, tn_tot, reduction='micro').item()
    precision = smp.metrics.precision(tp_tot, fp_tot, fn_tot, tn_tot, reduction='micro').item()
    accuracy = smp.metrics.accuracy(tp_tot, fp_tot, fn_tot, tn_tot, reduction='micro').item()

    return iou, recall, precision, accuracy

results_data = []

print(f"\nTversky Hyperparameter Tuning")
print(f"Testing Alphas: {TVERSKY_ALPHAS}")

for alpha in TVERSKY_ALPHAS:
    beta = 1.0 - alpha

    loss_name = f"Tversky (a={alpha}, b={beta:.2f})"

    print(f"Experiment: {loss_name}")

    model = get_hybrid_model(encoder_name="mit_b4")

    loss_fn = smp.losses.TverskyLoss(
        mode='binary',
        alpha=alpha,
        beta=beta,
        gamma=1.0
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

    best_iou = 0.0

    pbar = tqdm(range(1, EPOCHS + 1), desc=f"Training a={alpha}")
    for epoch in pbar:
        train_loss = train_epoch(model, train_loader, loss_fn, optimizer, DEVICE)

        iou, recall, precision, accuracy = evaluate_metrics(model, valid_loader, DEVICE)
        scheduler.step(iou)

        if iou > best_iou:
            best_iou = iou

        pbar.set_postfix({"Loss": f"{train_loss:.4f}", "IoU": f"{iou:.4f}"})

    print(f"Finished. Best IoU: {best_iou:.4f}")

    final_iou, final_recall, final_precision, final_accuracy = evaluate_metrics(model, valid_loader, DEVICE)

    results_data.append({
        "Model": "MitUnet",
        "Loss Function": "Tversky",
        "Alpha (FN penalty)": alpha,
        "Beta (FP penalty)": round(beta, 2),
        "IoU": round(final_iou, 4),
        "Recall": round(final_recall, 4),
        "Precision": round(final_precision, 4),
        "Accuracy": round(final_accuracy, 4)
    })

    del model, optimizer, scheduler, loss_fn
    torch.cuda.empty_cache()
    gc.collect()

df_results = pd.DataFrame(results_data)

df_results = df_results.sort_values(by="IoU", ascending=False).reset_index(drop=True)

print("\n\nTVERSKY PARAMETER RESULTS")
print(df_results.to_string())

try:
    df_results.to_excel("tversky_tuning_results.xlsx", index=False)
    print("\nResults saved to 'tversky_tuning_results.xlsx'")
except Exception as e:
    print(f"\nCould not save to Excel (install openpyxl). Saving to CSV. Error: {e}")
    df_results.to_csv("tversky_tuning_results.csv", index=False)

plt.figure(figsize=(10, 6))

plt.plot(df_results["Alpha (FN penalty)"], df_results["Recall"], marker='o', label='Recall', color='green')
plt.plot(df_results["Alpha (FN penalty)"], df_results["Precision"], marker='o', label='Precision', color='red')
plt.plot(df_results["Alpha (FN penalty)"], df_results["IoU"], marker='x', label='IoU', color='blue', linestyle='--')

plt.title(f"Alpha impact on metrics")
plt.xlabel("Alpha (FN)")
plt.ylabel("Score")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import torch
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

if torch.cuda.is_available():
    torch.cuda.empty_cache()

DEVICE = "cuda"
MODEL_NAME = 'MitUNet_finetune'
ENCODER_MITUNET = "mit_b4"
EPOCHS_MITUNET = 30
LR_MITUNET = 0.00001
BEST_MITUNET_PATH = "MitUNet_cubicasa-5k_a62_mit_b4_tversky_7606_20E.pth"

segformer_for_encoder = smp.Segformer(
    encoder_name=ENCODER_MITUNET,
    encoder_weights="imagenet",
)
transformer_encoder = segformer_for_encoder.encoder

model_mitunet = smp.Unet(
    encoder_name=ENCODER_MITUNET,
    encoder_weights=None,
    in_channels=3,
    classes=1,
    decoder_attention_type="scse"
)

model_mitunet.encoder = transformer_encoder
model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE)) # finetune
model_mitunet.to(DEVICE)

# loss_fn_mitunet = smp.losses.DiceLoss(mode='binary')
# loss_fn_mitunet = smp.losses.FocalLoss(mode='binary')
loss_fn_mitunet = smp.losses.LovaszLoss(mode='binary')
# loss_fn_mitunet = smp.losses.TverskyLoss(
#     mode='binary',
#     alpha=0.7,
#     beta=0.3,
#     gamma=1.0
# )

optimizer_mitunet = torch.optim.Adam(model_mitunet.parameters(), lr=LR_MITUNET)
lr_scheduler_mitunet = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_mitunet, mode='max', factor=0.5, patience=3
)

print(f"\nStart training {MODEL_NAME}")
max_iou_mitunet = 0

train_iou_history_su = []
valid_iou_history_su = []
train_loss_history_su = []
valid_loss_history_su = []
valid_fps_history_su = []
valid_vram_history_su = []

for epoch in range(1, EPOCHS_MITUNET + 1):
    print(f"\nEpoch {epoch}/{EPOCHS_MITUNET} ({MODEL_NAME})")

    train_loss, train_iou = train_epoch(model_mitunet, train_loader, loss_fn_mitunet, optimizer_mitunet, device=DEVICE)
    valid_loss, valid_iou, fps, vram = valid_epoch(model_mitunet, valid_loader, loss_fn_mitunet, device=DEVICE)

    train_loss_history_su.append(train_loss)
    valid_loss_history_su.append(valid_loss)
    train_iou_history_su.append(train_iou)
    valid_iou_history_su.append(valid_iou)
    valid_fps_history_su.append(fps)
    valid_vram_history_su.append(vram)

    lr_scheduler_mitunet.step(valid_iou)

    print(f"{MODEL_NAME} Train Loss: {train_loss:.4f} | {MODEL_NAME} Valid Loss: {valid_loss:.4f}")
    print(f"{MODEL_NAME} Train IoU: {train_iou:.4f} | {MODEL_NAME} Valid IoU: {valid_iou:.4f}")
    print(f"Speed: {fps:.2f} FPS | VRAM: {vram:.2f} MiB")

    if valid_iou > max_iou_mitunet:
        max_iou_mitunet = valid_iou
        BEST_MITUNET_PATH = f'{MODEL_NAME.lower()}_{ENCODER_MITUNET}_{str(loss_fn_mitunet).lower().split('loss', 1)[0]}_{f"{valid_iou:.4f}"[2:]}_{epoch}E.pth'

        if epoch > 3:
          torch.save(model_mitunet.state_dict(), BEST_MITUNET_PATH)
          print(f"{MODEL_NAME} model saved! New best validation IoU: {max_iou_mitunet:.4f}")

print(f"\nTraining {MODEL_NAME} is over. Best IoU: {max_iou_mitunet:.4f}")

plt.figure(figsize=(16, 12))
epochs_range = range(1, EPOCHS_MITUNET + 1)

# Loss
plt.subplot(2, 2, 1)
plt.plot(epochs_range, train_loss_history_su, label='Train Loss', marker='o')
plt.plot(epochs_range, valid_loss_history_su, label='Validation Loss', marker='o')
plt.title(f'Loss by epoch ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True)
plt.legend()

# IoU
plt.subplot(2, 2, 2)
plt.plot(epochs_range, train_iou_history_su, label='Train IoU', marker='o')
plt.plot(epochs_range, valid_iou_history_su, label='Validation IoU', marker='o')
plt.title(f'IoU by epoch ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('IoU Score', fontsize=12)
plt.grid(True)
plt.legend()

# FPS
plt.subplot(2, 2, 3)
plt.plot(epochs_range, valid_fps_history_su, label='Validation FPS', marker='o', color='green')
plt.title(f'FPS ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('FPS', fontsize=12)
plt.grid(True)
plt.legend()

# VRAM
plt.subplot(2, 2, 4)
plt.plot(epochs_range, valid_vram_history_su, label='Peak VRAM Usage', marker='o', color='purple')
plt.title(f'Peak VRAM Usage (MiB) ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('VRAM (MiB)', fontsize=12)
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from tqdm.auto import tqdm

def evaluate_model(model, loader, device):
    model.eval()

    total_tp, total_fp, total_fn, total_tn = 0, 0, 0, 0

    with torch.no_grad():
        pbar = tqdm(loader, desc="Metrics")
        for images, masks in pbar:
            images, masks = images.to(device), masks.to(device).unsqueeze(1).float()

            outputs = model(images)
            pred_probs = torch.sigmoid(outputs)
            pred_masks = (pred_probs > 0.5).float()

            tp, fp, fn, tn = smp.metrics.get_stats(pred_masks.long(), masks.long(), mode='binary')

            total_tp += tp.sum()
            total_fp += fp.sum()
            total_fn += fn.sum()
            total_tn += tn.sum()

    final_iou = smp.metrics.iou_score(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_recall = smp.metrics.recall(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_precision = smp.metrics.precision(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_accuracy = smp.metrics.accuracy(total_tp, total_fp, total_fn, total_tn, reduction='micro')

    return {
        "IoU": final_iou.item(),
        "Recall": final_recall.item(),
        "Precision": final_precision.item(),
        "Accuracy": final_accuracy.item()
    }

model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE))

print(f"Evaluating {BEST_MITUNET_PATH}")
metrics = evaluate_model(model_mitunet, valid_loader, device=DEVICE)

print("\nMetrics:")
print(f"  - IoU:       {metrics['IoU']:.4f}")
print(f"  - Recall:    {metrics['Recall']:.4f}")
print(f"  - Precision: {metrics['Precision']:.4f}")
print(f"  - Accuracy:  {metrics['Accuracy']:.4f}")

In [ ]:
import torch
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

if torch.cuda.is_available():
    torch.cuda.empty_cache()

MODEL_NAME = 'MitUNet_finetune'
ENCODER_MITUNET = "mit_b4"
EPOCHS_MITUNET = 30
LR_MITUNET = 0.00001
BEST_MITUNET_PATH = "MitUNet_cubicasa-5k_a62_mit_b4_tversky_7606_20E.pth"

segformer_for_encoder = smp.Segformer(
    encoder_name=ENCODER_MITUNET,
    encoder_weights="imagenet",
)
transformer_encoder = segformer_for_encoder.encoder

model_mitunet = smp.Unet(
    encoder_name=ENCODER_MITUNET,
    encoder_weights=None,
    in_channels=3,
    classes=1,
    decoder_attention_type="scse"
)

model_mitunet.encoder = transformer_encoder
model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE)) # finetune
model_mitunet.to(DEVICE)

# loss_fn_mitunet = smp.losses.DiceLoss(mode='binary')
loss_fn_mitunet = smp.losses.FocalLoss(mode='binary')
# loss_fn_mitunet = smp.losses.LovaszLoss(mode='binary')
# loss_fn_mitunet = smp.losses.TverskyLoss(
#     mode='binary',
#     alpha=0.7,
#     beta=0.3,
#     gamma=1.0
# )

optimizer_mitunet = torch.optim.Adam(model_mitunet.parameters(), lr=LR_MITUNET)
lr_scheduler_mitunet = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_mitunet, mode='max', factor=0.5, patience=3
)

print(f"\n Training {MODEL_NAME} ")
max_iou_mitunet = 0

train_iou_history_su = []
valid_iou_history_su = []
train_loss_history_su = []
valid_loss_history_su = []
valid_fps_history_su = []
valid_vram_history_su = []

for epoch in range(1, EPOCHS_MITUNET + 1):
    print(f"\n Epoch {epoch}/{EPOCHS_MITUNET} ({MODEL_NAME}) ")

    train_loss, train_iou = train_epoch(model_mitunet, train_loader, loss_fn_mitunet, optimizer_mitunet, device=DEVICE)
    valid_loss, valid_iou, fps, vram = valid_epoch(model_mitunet, valid_loader, loss_fn_mitunet, device=DEVICE)

    train_loss_history_su.append(train_loss)
    valid_loss_history_su.append(valid_loss)
    train_iou_history_su.append(train_iou)
    valid_iou_history_su.append(valid_iou)
    valid_fps_history_su.append(fps)
    valid_vram_history_su.append(vram)

    lr_scheduler_mitunet.step(valid_iou)

    print(f"{MODEL_NAME} Train Loss: {train_loss:.4f} | {MODEL_NAME} Valid Loss: {valid_loss:.4f}")
    print(f"{MODEL_NAME} Train IoU: {train_iou:.4f} | {MODEL_NAME} Valid IoU: {valid_iou:.4f}")
    print(f"Speed: {fps:.2f} FPS | VRAM: {vram:.2f} MiB")

    if valid_iou > max_iou_mitunet:
        max_iou_mitunet = valid_iou
        BEST_MITUNET_PATH = f'{MODEL_NAME.lower()}_{ENCODER_MITUNET}_{str(loss_fn_mitunet).lower().split('loss', 1)[0]}_{f"{valid_iou:.4f}"[2:]}_{epoch}E.pth'

        if epoch > 3:
          torch.save(model_mitunet.state_dict(), BEST_MITUNET_PATH)
          print(f"🏆 {MODEL_NAME} model saved! New best validation IoU: {max_iou_mitunet:.4f}")

print(f"\nTraining {MODEL_NAME} is over. Best IoU: {max_iou_mitunet:.4f}")

plt.figure(figsize=(16, 12))
epochs_range = range(1, EPOCHS_MITUNET + 1)

# Loss
plt.subplot(2, 2, 1)
plt.plot(epochs_range, train_loss_history_su, label='Train Loss', marker='o')
plt.plot(epochs_range, valid_loss_history_su, label='Validation Loss', marker='o')
plt.title(f'Loss by epoch ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True)
plt.legend()

# IoU
plt.subplot(2, 2, 2)
plt.plot(epochs_range, train_iou_history_su, label='Train IoU', marker='o')
plt.plot(epochs_range, valid_iou_history_su, label='Validation IoU', marker='o')
plt.title(f'IoU by epoch ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('IoU Score', fontsize=12)
plt.grid(True)
plt.legend()

# FPS
plt.subplot(2, 2, 3)
plt.plot(epochs_range, valid_fps_history_su, label='Validation FPS', marker='o', color='green')
plt.title(f'FPS ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('FPS', fontsize=12)
plt.grid(True)
plt.legend()

# VRAM
plt.subplot(2, 2, 4)
plt.plot(epochs_range, valid_vram_history_su, label='Peak VRAM Usage', marker='o', color='purple')
plt.title(f'Peak VRAM Usage (MiB) ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('VRAM (MiB)', fontsize=12)
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from tqdm.auto import tqdm

def evaluate_model(model, loader, device):
    model.eval()

    total_tp, total_fp, total_fn, total_tn = 0, 0, 0, 0

    with torch.no_grad():
        pbar = tqdm(loader, desc="Metrics")
        for images, masks in pbar:
            images, masks = images.to(device), masks.to(device).unsqueeze(1).float()

            outputs = model(images)
            pred_probs = torch.sigmoid(outputs)
            pred_masks = (pred_probs > 0.5).float()

            tp, fp, fn, tn = smp.metrics.get_stats(pred_masks.long(), masks.long(), mode='binary')

            total_tp += tp.sum()
            total_fp += fp.sum()
            total_fn += fn.sum()
            total_tn += tn.sum()

    final_iou = smp.metrics.iou_score(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_recall = smp.metrics.recall(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_precision = smp.metrics.precision(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_accuracy = smp.metrics.accuracy(total_tp, total_fp, total_fn, total_tn, reduction='micro')

    return {
        "IoU": final_iou.item(),
        "Recall": final_recall.item(),
        "Precision": final_precision.item(),
        "Accuracy": final_accuracy.item()
    }

model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE))

print(f"Evaluating {BEST_MITUNET_PATH}")
metrics = evaluate_model(model_mitunet, valid_loader, device=DEVICE)

print("\nMetrics:")
print(f"  - IoU:       {metrics['IoU']:.4f}")
print(f"  - Recall:    {metrics['Recall']:.4f}")
print(f"  - Precision: {metrics['Precision']:.4f}")
print(f"  - Accuracy:  {metrics['Accuracy']:.4f}")

In [ ]:
import torch
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

if torch.cuda.is_available():
    torch.cuda.empty_cache()

MODEL_NAME = 'MitUNet_finetune'
ENCODER_MITUNET = "mit_b4"
EPOCHS_MITUNET = 30
LR_MITUNET = 0.00001
BEST_MITUNET_PATH = "MitUNet_cubicasa-5k_a62_mit_b4_tversky_7606_20E.pth"

segformer_for_encoder = smp.Segformer(
    encoder_name=ENCODER_MITUNET,
    encoder_weights="imagenet",
)
transformer_encoder = segformer_for_encoder.encoder

model_mitunet = smp.Unet(
    encoder_name=ENCODER_MITUNET,
    encoder_weights=None,
    in_channels=3,
    classes=1,
    decoder_attention_type="scse"
)

model_mitunet.encoder = transformer_encoder
model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE)) # finetune
model_mitunet.to(DEVICE)

loss_fn_mitunet = smp.losses.DiceLoss(mode='binary')
# loss_fn_mitunet = smp.losses.FocalLoss(mode='binary')
# loss_fn_mitunet = smp.losses.LovaszLoss(mode='binary')
# loss_fn_mitunet = smp.losses.TverskyLoss(
#     mode='binary',
#     alpha=0.7,
#     beta=0.3,
#     gamma=1.0
# )

optimizer_mitunet = torch.optim.Adam(model_mitunet.parameters(), lr=LR_MITUNET)
lr_scheduler_mitunet = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_mitunet, mode='max', factor=0.5, patience=3
)

print(f"\nTraining {MODEL_NAME}")
max_iou_mitunet = 0

train_iou_history_su = []
valid_iou_history_su = []
train_loss_history_su = []
valid_loss_history_su = []
valid_fps_history_su = []
valid_vram_history_su = []

for epoch in range(1, EPOCHS_MITUNET + 1):
    print(f"\nEpoch {epoch}/{EPOCHS_MITUNET} ({MODEL_NAME})")

    train_loss, train_iou = train_epoch(model_mitunet, train_loader, loss_fn_mitunet, optimizer_mitunet, device=DEVICE)
    valid_loss, valid_iou, fps, vram = valid_epoch(model_mitunet, valid_loader, loss_fn_mitunet, device=DEVICE)

    train_loss_history_su.append(train_loss)
    valid_loss_history_su.append(valid_loss)
    train_iou_history_su.append(train_iou)
    valid_iou_history_su.append(valid_iou)
    valid_fps_history_su.append(fps)
    valid_vram_history_su.append(vram)

    lr_scheduler_mitunet.step(valid_iou)

    print(f"{MODEL_NAME} Train Loss: {train_loss:.4f} | {MODEL_NAME} Valid Loss: {valid_loss:.4f}")
    print(f"{MODEL_NAME} Train IoU: {train_iou:.4f} | {MODEL_NAME} Valid IoU: {valid_iou:.4f}")
    print(f"Speed: {fps:.2f} FPS | VRAM: {vram:.2f} MiB")

    if valid_iou > max_iou_mitunet:
        max_iou_mitunet = valid_iou
        BEST_MITUNET_PATH = f'{MODEL_NAME.lower()}_{ENCODER_MITUNET}_{str(loss_fn_mitunet).lower().split('loss', 1)[0]}_{f"{valid_iou:.4f}"[2:]}_{epoch}E.pth'

        if epoch > 3:
          torch.save(model_mitunet.state_dict(), BEST_MITUNET_PATH)
          print(f"{MODEL_NAME} model saved! New best validation IoU: {max_iou_mitunet:.4f}")


print(f"\nTraining {MODEL_NAME} is over. Best IoU: {max_iou_mitunet:.4f}")

plt.figure(figsize=(16, 12))
epochs_range = range(1, EPOCHS_MITUNET + 1)

# Loss
plt.subplot(2, 2, 1)
plt.plot(epochs_range, train_loss_history_su, label='Train Loss', marker='o')
plt.plot(epochs_range, valid_loss_history_su, label='Validation Loss', marker='o')
plt.title(f'Loss by epoch ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True)
plt.legend()

# IoU
plt.subplot(2, 2, 2)
plt.plot(epochs_range, train_iou_history_su, label='Train IoU', marker='o')
plt.plot(epochs_range, valid_iou_history_su, label='Validation IoU', marker='o')
plt.title(f'IoU by epoch ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('IoU Score', fontsize=12)
plt.grid(True)
plt.legend()

# FPS
plt.subplot(2, 2, 3)
plt.plot(epochs_range, valid_fps_history_su, label='Validation FPS', marker='o', color='green')
plt.title(f'FPS ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('FPS', fontsize=12)
plt.grid(True)
plt.legend()

# VRAM
plt.subplot(2, 2, 4)
plt.plot(epochs_range, valid_vram_history_su, label='Peak VRAM Usage', marker='o', color='purple')
plt.title(f'Peak VRAM Usage (MiB) ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('VRAM (MiB)', fontsize=12)
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

from tqdm.auto import tqdm

def evaluate_model(model, loader, device):
    model.eval()

    total_tp, total_fp, total_fn, total_tn = 0, 0, 0, 0

    with torch.no_grad():
        pbar = tqdm(loader, desc="Metrics")
        for images, masks in pbar:
            images, masks = images.to(device), masks.to(device).unsqueeze(1).float()

            outputs = model(images)
            pred_probs = torch.sigmoid(outputs)
            pred_masks = (pred_probs > 0.5).float()

            tp, fp, fn, tn = smp.metrics.get_stats(pred_masks.long(), masks.long(), mode='binary')

            total_tp += tp.sum()
            total_fp += fp.sum()
            total_fn += fn.sum()
            total_tn += tn.sum()

    final_iou = smp.metrics.iou_score(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_recall = smp.metrics.recall(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_precision = smp.metrics.precision(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_accuracy = smp.metrics.accuracy(total_tp, total_fp, total_fn, total_tn, reduction='micro')

    return {
        "IoU": final_iou.item(),
        "Recall": final_recall.item(),
        "Precision": final_precision.item(),
        "Accuracy": final_accuracy.item()
    }

model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE))

print(f"Evaluating {BEST_MITUNET_PATH}")
metrics = evaluate_model(model_mitunet, valid_loader, device=DEVICE)

print("\nMetrics:")
print(f"  - IoU:       {metrics['IoU']:.4f}")
print(f"  - Recall:    {metrics['Recall']:.4f}")
print(f"  - Precision: {metrics['Precision']:.4f}")
print(f"  - Accuracy:  {metrics['Accuracy']:.4f}")

In [ ]:
import torch
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

if torch.cuda.is_available():
    torch.cuda.empty_cache()

MODEL_NAME = 'MitUNet_finetune'
ENCODER_MITUNET = "mit_b4"
EPOCHS_MITUNET = 30
LR_MITUNET = 0.00001
BEST_MITUNET_PATH = "MitUNet_cubicasa-5k_a62_mit_b4_tversky_7606_20E.pth"

segformer_for_encoder = smp.Segformer(
    encoder_name=ENCODER_MITUNET,
    encoder_weights="imagenet",
)
transformer_encoder = segformer_for_encoder.encoder

model_mitunet = smp.Unet(
    encoder_name=ENCODER_MITUNET,
    encoder_weights=None,
    in_channels=3,
    classes=1,
    decoder_attention_type="scse"
)

model_mitunet.encoder = transformer_encoder
model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE)) # finetune
model_mitunet.to(DEVICE)

# loss_fn_mitunet = smp.losses.DiceLoss(mode='binary')
# loss_fn_mitunet = smp.losses.FocalLoss(mode='binary')
loss_fn_mitunet = smp.losses.LovaszLoss(mode='binary')
# loss_fn_mitunet = smp.losses.TverskyLoss(
#     mode='binary',
#     alpha=0.6,
#     beta=0.4,
#     gamma=1.0
# )

optimizer_mitunet = torch.optim.Adam(model_mitunet.parameters(), lr=LR_MITUNET)
lr_scheduler_mitunet = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer_mitunet, mode='max', factor=0.5, patience=3
)

print(f"\nTraining {MODEL_NAME}")
max_iou_mitunet = 0

train_iou_history_su = []
valid_iou_history_su = []
train_loss_history_su = []
valid_loss_history_su = []
valid_fps_history_su = []
valid_vram_history_su = []

for epoch in range(1, EPOCHS_MITUNET + 1):
    print(f"\n Epoch {epoch}/{EPOCHS_MITUNET} ({MODEL_NAME}) ")

    train_loss, train_iou = train_epoch(model_mitunet, train_loader, loss_fn_mitunet, optimizer_mitunet, device=DEVICE)
    valid_loss, valid_iou, fps, vram = valid_epoch(model_mitunet, valid_loader, loss_fn_mitunet, device=DEVICE)

    train_loss_history_su.append(train_loss)
    valid_loss_history_su.append(valid_loss)
    train_iou_history_su.append(train_iou)
    valid_iou_history_su.append(valid_iou)
    valid_fps_history_su.append(fps)
    valid_vram_history_su.append(vram)

    lr_scheduler_mitunet.step(valid_iou)

    print(f"{MODEL_NAME} Train Loss: {train_loss:.4f} | {MODEL_NAME} Valid Loss: {valid_loss:.4f}")
    print(f"{MODEL_NAME} Train IoU: {train_iou:.4f} | {MODEL_NAME} Valid IoU: {valid_iou:.4f}")
    print(f"Speed: {fps:.2f} FPS | VRAM: {vram:.2f} MiB")

    if valid_iou > max_iou_mitunet:
        max_iou_mitunet = valid_iou
        BEST_MITUNET_PATH = f'{MODEL_NAME.lower()}_{ENCODER_MITUNET}_{str(loss_fn_mitunet).lower().split('loss', 1)[0]}_{f"{valid_iou:.4f}"[2:]}_{epoch}E.pth'

        if epoch > 3:
          torch.save(model_mitunet.state_dict(), BEST_MITUNET_PATH)
          print(f"🏆 {MODEL_NAME} model saved! New best validation IoU: {max_iou_mitunet:.4f}")

print(f"\nTraining {MODEL_NAME} is over. Best IoU: {max_iou_mitunet:.4f}")


# Loss
plt.subplot(2, 2, 1)
plt.plot(epochs_range, train_loss_history_su, label='Train Loss', marker='o')
plt.plot(epochs_range, valid_loss_history_su, label='Validation Loss', marker='o')
plt.title(f'Loss by epoch ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True)
plt.legend()

# IoU
plt.subplot(2, 2, 2)
plt.plot(epochs_range, train_iou_history_su, label='Train IoU', marker='o')
plt.plot(epochs_range, valid_iou_history_su, label='Validation IoU', marker='o')
plt.title(f'IoU by epoch ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('IoU Score', fontsize=12)
plt.grid(True)
plt.legend()

# FPS
plt.subplot(2, 2, 3)
plt.plot(epochs_range, valid_fps_history_su, label='Validation FPS', marker='o', color='green')
plt.title(f'FPS ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('FPS', fontsize=12)
plt.grid(True)
plt.legend()

# VRAM
plt.subplot(2, 2, 4)
plt.plot(epochs_range, valid_vram_history_su, label='Peak VRAM Usage', marker='o', color='purple')
plt.title(f'Peak VRAM Usage (MiB) ({MODEL_NAME})', fontsize=16)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('VRAM (MiB)', fontsize=12)
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

from tqdm.auto import tqdm

def evaluate_model(model, loader, device):
    model.eval()

    total_tp, total_fp, total_fn, total_tn = 0, 0, 0, 0

    with torch.no_grad():
        pbar = tqdm(loader, desc="Metrics")
        for images, masks in pbar:
            images, masks = images.to(device), masks.to(device).unsqueeze(1).float()

            outputs = model(images)
            pred_probs = torch.sigmoid(outputs)
            pred_masks = (pred_probs > 0.5).float()

            tp, fp, fn, tn = smp.metrics.get_stats(pred_masks.long(), masks.long(), mode='binary')

            total_tp += tp.sum()
            total_fp += fp.sum()
            total_fn += fn.sum()
            total_tn += tn.sum()

    final_iou = smp.metrics.iou_score(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_recall = smp.metrics.recall(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_precision = smp.metrics.precision(total_tp, total_fp, total_fn, total_tn, reduction='micro')
    final_accuracy = smp.metrics.accuracy(total_tp, total_fp, total_fn, total_tn, reduction='micro')

    return {
        "IoU": final_iou.item(),
        "Recall": final_recall.item(),
        "Precision": final_precision.item(),
        "Accuracy": final_accuracy.item()
    }

model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE))

print(f"Evaluating {BEST_MITUNET_PATH}")
metrics = evaluate_model(model_mitunet, valid_loader, device=DEVICE)

print("\nMetrics:")
print(f"  - IoU:       {metrics['IoU']:.4f}")
print(f"  - Recall:    {metrics['Recall']:.4f}")
print(f"  - Precision: {metrics['Precision']:.4f}")
print(f"  - Accuracy:  {metrics['Accuracy']:.4f}")

In [ ]:
import torch
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from pycocotools.coco import COCO
import cv2
import numpy as np
import random
import os
import matplotlib.pyplot as plt

inference_transforms = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

def predict(model, image_rgb):
    model.eval()
    augmented = inference_transforms(image=image_rgb)
    image_tensor = augmented['image'].unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        logits = model(image_tensor)
        probs = torch.sigmoid(logits)
        mask = (probs > 0.5).float()

    result_mask = mask.cpu().squeeze().numpy()
    return result_mask

segformer_for_encoder = smp.Segformer(
    encoder_name="mit_b4",
    encoder_weights="imagenet",
)
transformer_encoder = segformer_for_encoder.encoder

model_mitunet = smp.Unet(
    encoder_name="mit_b4",
    encoder_weights=None,
    in_channels=3,
    classes=1,
    decoder_attention_type="scse",
)

model_mitunet.encoder = transformer_encoder
model_mitunet.to(DEVICE)
model_mitunet.eval()

valid_dataset_raw = COCOSegmentationDataset(valid_dir, valid_annot, transforms=None)

random_idx = random.randint(0, len(valid_dataset_raw) - 1)
original_image, ground_truth_mask = valid_dataset_raw[random_idx]

segunet_prediction = predict(model_mitunet, original_image)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(original_image)
axes[0].set_title(f"Original\n(Index: {random_idx})")
axes[0].axis('off')

axes[1].imshow(ground_truth_mask, cmap='gray')
axes[1].set_title("Ground Truth")

axes[2].imshow(segunet_prediction, cmap='gray')
axes[2].set_title("MitUnet Prediction")

plt.tight_layout()
plt.show()

In [ ]:
import torch
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

BEST_MITUNET_PATH = "mitunet_finetune_a6_mit_b4_tversky_8864_28E.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

inference_transforms = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

segformer_for_encoder = smp.Segformer(encoder_name="mit_b4", encoder_weights="imagenet")
transformer_encoder = segformer_for_encoder.encoder
model_mitunet = smp.Unet(
    encoder_name="mit_b4", encoder_weights=None, in_channels=3, classes=1, decoder_attention_type="scse"
)
model_mitunet.encoder = transformer_encoder
model_mitunet.load_state_dict(torch.load(BEST_MITUNET_PATH, map_location=DEVICE))
model_mitunet.to(DEVICE)
model_mitunet.eval()

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
def predict(model, image_rgb):
    model.eval()
    augmented = inference_transforms(image=image_rgb)
    image_tensor = augmented['image'].unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        logits = model(image_tensor)
        probs = torch.sigmoid(logits)
        mask = (probs > 0.5).float()

    result_mask = mask.cpu().squeeze().numpy()
    return result_mask


for file_name in uploaded.keys():
    print(f"\nProcessing: {file_name}")

    image_bytes = uploaded[file_name]
    nparr = np.frombuffer(image_bytes, np.uint8)
    image_bgr = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    original_image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    segunet_prediction = predict(model_mitunet, original_image)

    fig, axes = plt.subplots(1, 2, figsize=(24, 6))

    axes[0].imshow(original_image)
    axes[0].set_title("Original")
    axes[0].axis('off')

    axes[1].imshow(segunet_prediction, cmap='gray')
    axes[1].set_title("MitUNet")
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()
    break